In [1]:
# package imports
import base64
from dotenv import load_dotenv
import numpy as np
import os
import pandas as pd
import requests

In [2]:
# read environment variables
load_dotenv()
# client id and secret
SPOTIFY_API_CLIENT_ID = os.getenv('SPOTIFY_API_CLIENT_ID')
SPOTIFY_API_CLIENT_SECRET = os.getenv('SPOTIFY_API_CLIENT_SECRET')
# refresh token
SPOTIFY_REFRESH_TOKEN = os.getenv('SPOTIFY_REFRESH_TOKEN')
# api urls
SPOTIFY_API_BASE_URL = os.getenv('SPOTIFY_API_BASE_URL')
SPOTIFY_API_TOKEN_URL = os.getenv('SPOTIFY_API_TOKEN_URL')
# redirect uri
SPOTIFY_API_REDIRECT_URI = os.getenv('SPOTIFY_API_REDIRECT_URI')
# api paths
USER_TOP_TRACKS_ENDPOINT = os.getenv('USER_TOP_TRACKS_ENDPOINT')

In [15]:
def get_access_token():
    # Construct the request URL with query parameters
    url = f"{SPOTIFY_API_TOKEN_URL}?grant_type=refresh_token&refresh_token={SPOTIFY_REFRESH_TOKEN}"

    # Encode the client ID and client secret in base64
    client_credentials = f"{SPOTIFY_API_CLIENT_ID}:{SPOTIFY_API_CLIENT_SECRET}"
    base64_credentials = base64.b64encode(client_credentials.encode()).decode()

    # Send a POST request to the token endpoint
    response = requests.post(url, headers={
        "Authorization": f"Basic {base64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    })

    # Parse the JSON response and extract the access token
    data = response.json()
    access_token = data.get("access_token")

    return access_token

In [35]:
res = requests.get(f"{SPOTIFY_API_BASE_URL}{USER_TOP_TRACKS_ENDPOINT}", headers = {
    "Authorization": f"Bearer {get_access_token()}"
}).json()

top_tracks = []

for i, item in enumerate(res['items']):
    artists = [artist['name'] for artist in item['artists']]
    top_tracks.append(f"{i + 1}. {item['name']} - {', '.join(artists)}")

for track in top_tracks:
    print('\n', track)


 1. Decatur - 6LACK

 2. Georgia Peach - Quinn XCII

 3. Lady Writer - Dire Straits

 4. Hands On You - Austin George

 5. Believe - NORRA, lostinspace

 6. Any Colour You Like - Pink Floyd

 7. Assumptions - Kaytranada Edit - Sam Gellaitry, KAYTRANADA

 8. Sweet Virginia - The Rolling Stones

 9. Coastal Cat - Tez Cadey

 10. Shine On You Crazy Diamond (Pts. 1-5) - Pink Floyd

 11. Try Me - The Weeknd

 12. Speak to Me - Pink Floyd

 13. KANTE (feat. Fave) - Davido, Fave

 14. Jersey Giant - Elle King

 15. 4EVA (feat. Pharrell Williams) - KAYTRAMINÉ, Aminé, KAYTRANADA, Pharrell Williams

 16. High Noon - Kota the Friend, Statik Selektah

 17. Being Me - Quinn XCII

 18. Bad Boy Boogie - AC/DC

 19. Running In Circles - Mat Kearney

 20. So Long - Shallou, Origami Human
